In [36]:
import pandas as pd
import requests
import json

try:
    from routing_module import *
except:
    import sys
    sys.path.append('/home/happy956/')
    from routing_module import *

### 1. previous bus station

In [11]:
prev_bus_station = [
    {
        'start_station': '걸포마루공원',
        'end_station': '걸포북변역.국민연금공단',
        'start_lat': 37.632597,
        'start_lon': 126.7035,
        'end_lat': 37.631468,
        'end_lon': 126.705672,
        'direction': 1
    },
    {
        'start_station': '걸포북변역.국민연금공단',
        'end_station': '풍무역',
        'start_lat': 37.631468,
        'start_lon': 126.705672,
        'end_lat': 37.612075,
        'end_lon': 126.732771,
        'direction': 1
    },
    {
        'start_station': '풍무역',
        'end_station': '고촌역',
        'start_lat': 37.612075,
        'start_lon': 126.732771,
        'end_lat': 37.6004,
        'end_lon': 126.770752,
        'direction': 1
    },
    {
        'start_station': '고촌역',
        'end_station': '개화역광역환승센터',
        'start_lat': 37.6004,
        'start_lon': 126.770752,
        'end_lat': 37.578385,
        'end_lon': 126.797538,
        'direction': 1
    },
    {
        'start_station': '개화역광역환승센터',
        'end_station': '롯데몰.김포공항역',
        'start_lat': 37.578385,
        'start_lon': 126.797538,
        'end_lat': 37.56234,
        'end_lon': 126.803093,
        'direction': 1
    },
    {   
        'start_station': '롯데몰.김포공항역',
        'end_station': '개화역광역환승센터',
        'start_lat': 37.56234,
        'start_lon': 126.803093,
        'end_lat': 37.578229,
        'end_lon': 126.79814,
        'direction': 2
    },
    {   
        'start_station': '개화역광역환승센터',
        'end_station': '고촌역',
        'start_lat': 37.578229,
        'start_lon': 126.79814,
        'end_lat': 37.601408,
        'end_lon': 126.770414,
        'direction': 2
    },
    {
        'start_station': '고촌역',
        'end_station': '풍무역',
        'start_lat': 37.601408,
        'start_lon': 126.770414,
        'end_lat': 37.611934,
        'end_lon': 126.734239,
        'direction': 2
    },
    {
        'start_station': '풍무역',
        'end_station': '걸포북변역.국민연금공단',
        'start_lat': 37.611934,
        'start_lon': 126.734239,
        'end_lat': 37.631736,
        'end_lon': 126.70624,
        'direction': 2
    },
    {   
        'start_station': '롯데몰.김포공항역',
        'end_station': '개화역광역환승센터',
        'start_lat': 37.56234,
        'start_lon': 126.803093,
        'end_lat': 37.578229,
        'end_lon': 126.79814,
        'direction': 2
    }
]

In [16]:
with open('data/prev_bus/prev_bus_station.json', 'w', encoding='utf8') as f:
    json.dump(prev_bus_station, f, indent=4, ensure_ascii=False)

### 2. previous bus path

In [14]:
naver_client_id = 'egr8gx3fw8'
naver_client_secret = 'gwRtvccDkziFp9eQJh8Mg4wAhdjnvAbIs7wjGW3c'

naver_api = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?'
naver_headers = {'X-NCP-APIGW-API-KEY-ID': naver_client_id, 'X-NCP-APIGW-API-KEY': naver_client_secret}

In [21]:
def get_route():
    with open(f'data/prev_bus/prev_bus_station.json', 'r', encoding='utf8') as f:
        data = json.load(f)
    
    total_data = []
    for d in data:
        start_station, end_station = d['start_station'], d['end_station']
        start_lat, start_lon = d['start_lat'], d['start_lon']
        end_lat, end_lon = d['end_lat'], d['end_lon']
        
        url = f'{naver_api}start={start_lon},{start_lat}&goal={end_lon},{end_lat}'
        res = requests.get(url, headers=naver_headers).json()
        if res.get('code', 1) != 0: return {'code': 0}
        
        summary = res['route']['traoptimal'][0]['summary']
        route = res['route']['traoptimal'][0]['path']
        duration = summary['duration'] / 60000
        
        total_data.append({
            'bus_type': 'prev_bus',
            'start_station': start_station,
            'end_station': end_station,
            'route': route,
            'duration': duration,
            'direction': d['direction'],
        })
    return total_data

In [22]:
total_path = get_route()

with open('data/prev_bus/prev_bus_path.json', 'w', encoding='utf8') as f:
    json.dump(total_path, f, ensure_ascii=False)

### 3. previous bus trip & timestamp

In [25]:
with open('data/prev_bus/prev_bus_path.json', 'r', encoding='utf8') as f:
    path = json.load(f)

### previous bus schedule
prev_bus_schedule = pd.read_csv('data/prev_bus/prev_bus_schedule.csv')

In [43]:
def total_get_trip():
    total_trip = []
    for idx, prev_bus in prev_bus_schedule.groupby('버스_id'):
        prev_bus_trip = []
        prev_bus_timestamp = []
        
        for idx_, row in prev_bus.reset_index(drop=True).iterrows():
            start_station, end_station = row['출발정류장'], row['도착정류장']
            start_time_str = row['출발시간'].split(':')
            start_time = int(start_time_str[0]) * 60 + int(start_time_str[1])
            
            route_data = list(filter(lambda x: x['start_station'] == start_station and x['end_station'] == end_station, path))[0]
            routes, duration = route_data['route'], route_data['duration']
            
            if idx_ == 0:
                timestamp = get_timestamp(routes, start_time, duration)
            else:
                timestamp = get_timestamp(routes, timestamp[-1], duration)
                routes, timestamp = routes[1:], timestamp[1:]
                
            prev_bus_trip += routes
            prev_bus_timestamp += timestamp
        
        total_trip.append({
            'bus_id': idx,
            'trip': prev_bus_trip,
            'timestamp': prev_bus_timestamp,
            'direction': route_data['direction']
        })
    return total_trip

In [44]:
total_trip = total_get_trip()

with open('data/prev_bus/prev_bus_trip.json', 'w') as f:
    json.dump(total_trip, f)